<a href="https://colab.research.google.com/github/andreaco/SpokenDigitClassification/blob/master/SpokenDigitClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoken Digit Classification

In [0]:
import numpy as np
import re
import os
import librosa

In [1]:
! git clone https://github.com/Jakobovski/free-spoken-digit-dataset
#! pip install -r free-spoken-digit-dataset/pip_requirements.txt # probably not needed

Cloning into 'free-spoken-digit-dataset'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 3166 (delta 3), reused 5 (delta 0), pack-reused 3157
Receiving objects: 100% (3166/3166), 23.92 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [31]:
# recordings directory
directory = 'free-spoken-digit-dataset/recordings/'
# regexp matching {digitLabel}_{speakerName}_{index}.wav
regexp = re.compile('([0-9])+_([a-z]+)_([0-9]+).wav') 

digitLabels = []
speakerNames = []
indexes = []
audios = []
for f in os.listdir(directory):
  if f.endswith(".wav"):
    digitLabels. append(regexp.search(f).group(1))
    speakerNames.append(regexp.search(f).group(2))
    indexes. append(int(regexp.search(f).group(3)))

digitLabels = np.array(digitLabels)
speakerNames = np.array(speakerNames)
indexes = np.array(indexes)
audio = np.array(audio)

print("Elements in train set: {}".format(np.sum(indexes>4)))
print("Elements in test set: {}".format(np.sum(indexes<=4)))

AttributeError: ignored

In [17]:
infos_list[:,2]


array(['12', '23', '1', ..., '8', '17', '14'], dtype='<U8')

In [30]:
audio.shape

(2000, 3)